**Import Required Libraries**

In [0]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable

**Load Project Utilities & Initialize Notebook Widgets**

In [0]:
%run /Workspace/consolidated_pipeline/1_setup/utilities

In [0]:
print(bronze_schema, silver_schema, gold_schema)

bronze silver gold


In [0]:
dbutils.widgets.text("catalog", "fmcg", "Catalog")
dbutils.widgets.text("data_source", "customers", "Data Source")

catalog = dbutils.widgets.get("catalog")
data_source = dbutils.widgets.get("data_source")

base_path = f's3://sportsbar-final/{data_source}/*.csv'
print(base_path)

s3://sportsbar-final/customers/*.csv


## Bronze

In [0]:
df = (
    spark.read.format("csv")
        .option("header", True)
        .option("inferSchema", True)
        .load(base_path)
        .withColumn("read_timestamp", F.current_timestamp())
        .select("*", "_metadata.file_name", "_metadata.file_size")
)

In [0]:
# print check data type
df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- read_timestamp: timestamp (nullable = false)
 |-- file_name: string (nullable = false)
 |-- file_size: long (nullable = false)



In [0]:
display(df.limit(10))

customer_id,customer_name,city,read_timestamp,file_name,file_size
789201,FitFuel Market,Bengaluru,2025-11-23T11:52:51.119Z,customers.csv,1404
789202,FitFuel Market,Hyderabad,2025-11-23T11:52:51.119Z,customers.csv,1404
789203,FitFuel Market,New Delhi,2025-11-23T11:52:51.119Z,customers.csv,1404
789301,Athlete's Choice Store,Bengaluru,2025-11-23T11:52:51.119Z,customers.csv,1404
789303,Athlete's Choice Store,New Delhi,2025-11-23T11:52:51.119Z,customers.csv,1404
789101,Endurance Foods,Bengalore,2025-11-23T11:52:51.119Z,customers.csv,1404
789102,Endurance Foods,Hyderabad,2025-11-23T11:52:51.119Z,customers.csv,1404
789103,Endurance Foods,New Delhi,2025-11-23T11:52:51.119Z,customers.csv,1404
789121,HydroBoost Nutrition,Hyderabad,2025-11-23T11:52:51.119Z,customers.csv,1404
789122,HydroBoost Nutrition,New Delhi,2025-11-23T11:52:51.119Z,customers.csv,1404


In [0]:
df.write\
 .format("delta") \
 .option("delta.enableChangeDataFeed", "true") \
 .mode("overwrite") \
 .saveAsTable(f"{catalog}.{bronze_schema}.{data_source}")

## Silver

In [0]:
df_bronze = spark.sql(f"SELECT * FROM {catalog}.{bronze_schema}.{data_source};")
df_bronze.show(10)

+-----------+--------------------+---------+--------------------+-------------+---------+
|customer_id|       customer_name|     city|      read_timestamp|    file_name|file_size|
+-----------+--------------------+---------+--------------------+-------------+---------+
|     789201|      FitFuel Market|Bengaluru|2025-11-23 11:53:...|customers.csv|     1404|
|     789202|      FitFuel Market|Hyderabad|2025-11-23 11:53:...|customers.csv|     1404|
|     789203|      FitFuel Market|New Delhi|2025-11-23 11:53:...|customers.csv|     1404|
|     789301|Athlete's Choice ...|Bengaluru|2025-11-23 11:53:...|customers.csv|     1404|
|     789303|Athlete's Choice ...|New Delhi|2025-11-23 11:53:...|customers.csv|     1404|
|     789101|     Endurance Foods|Bengalore|2025-11-23 11:53:...|customers.csv|     1404|
|     789102|     Endurance Foods|Hyderabad|2025-11-23 11:53:...|customers.csv|     1404|
|     789103|     Endurance Foods|New Delhi|2025-11-23 11:53:...|customers.csv|     1404|
|     7891

In [0]:
df_bronze.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- read_timestamp: timestamp (nullable = true)
 |-- file_name: string (nullable = true)
 |-- file_size: long (nullable = true)



**Transformations**

- 1: Drop Duplicates

In [0]:


df_duplicates = df_bronze.groupBy("customer_id").count().filter(F.col("count") > 1)
display(df_duplicates)

customer_id,count
789321,2
789503,2
789522,2
789603,2


In [0]:
print('Rows before duplicates dropped: ', df_bronze.count())
df_silver = df_bronze.dropDuplicates(['customer_id'])
print('Rows after duplicates dropped: ', df_silver.count())

Rows before duplicates dropped:  39
Rows after duplicates dropped:  35


- 2: Trim spaces in customer name

In [0]:
# check those values
display(
    df_silver.filter(F.col("customer_name") != F.trim(F.col("customer_name")))
)

customer_id,customer_name,city,read_timestamp,file_name,file_size
789121,HydroBoost Nutrition,Hyderabad,2025-11-23T11:53:03.643Z,customers.csv,1404
789401,SprintX nutrition,Bengaluru,2025-11-23T11:53:03.643Z,customers.csv,1404
789420,ZenAthlete foods,null,2025-11-23T11:53:03.643Z,customers.csv,1404
789421,ZenAthlete Foods,Hyderbad,2025-11-23T11:53:03.643Z,customers.csv,1404
789521,PrimeFuel Nutrition,null,2025-11-23T11:53:03.643Z,customers.csv,1404
789702,StaminaX Store,Hyderabad,2025-11-23T11:53:03.643Z,customers.csv,1404


In [0]:
## remove those trim values

df_silver = df_silver.withColumn(
    "customer_name",
    F.trim(F.col("customer_name"))
)

In [0]:
# # Sanity Check

# # check those values
# display(
#     df_silver.filter(F.col("customer_name") != F.trim(F.col("customer_name")))
# )


- 3: Data Quality Fix: Correcting City Typos

In [0]:
df_silver.select('city').distinct().show()

+----------+
|      city|
+----------+
| Bengaluru|
| Hyderabad|
| New Delhi|
| Bengalore|
|Hyderabadd|
|      NULL|
|  Hyderbad|
| NewDelhee|
|  NewDelhi|
|Bengaluruu|
|  NewDheli|
+----------+



In [0]:
# # typo dictionary
# city_typos = {
#     'Bengaluru': ['Bengaluruu', 'Bengaluruu', 'Bengalore'],
#     'Hyderabad': ['Hyderabadd', 'Hyderbad'],
#     'New Delhi': ['NewDelhi', 'NewDheli', 'NewDelhee']
# }

# typos → correct names
city_mapping = {
    'Bengaluruu': 'Bengaluru',
    'Bengalore': 'Bengaluru',

    'Hyderabadd': 'Hyderabad',
    'Hyderbad': 'Hyderabad',

    'NewDelhi': 'New Delhi',
    'NewDheli': 'New Delhi',
    'NewDelhee': 'New Delhi'
}


allowed = ["Bengaluru", "Hyderabad", "New Delhi"]

df_silver = (
    df_silver
    .replace(city_mapping, subset=["city"])
    .withColumn(
        "city",
        F.when(F.col("city").isNull(), None)
         .when(F.col("city").isin(allowed), F.col("city"))
         .otherwise(None)
    )
)

In [0]:
# Sanity check
df_silver.select('city').distinct().show()

+---------+
|     city|
+---------+
|Bengaluru|
|Hyderabad|
|New Delhi|
|     NULL|
+---------+



- 4: Fix Title-Casing Issue

In [0]:
df_silver.select('customer_name').distinct().show()

+--------------------+
|       customer_name|
+--------------------+
|      FitFuel Market|
|Athlete's Choice ...|
|     Endurance Foods|
|HydroBoost Nutrition|
|MacroBite Superfoods|
|MacroBite superfoods|
|      PowerSnack Hub|
|      PowerSnack hub|
|   SprintX nutrition|
|   SprintX Nutrition|
|    ZenAthlete foods|
|    ZenAthlete Foods|
|Peak performance ...|
|Peak Performance ...|
| PrimeFuel Nutrition|
|       Recovery Lane|
|      StaminaX Store|
|EliteAthlete Nutr...|
|      GamePlan Foods|
|   Champion's choice|
+--------------------+
only showing top 20 rows


In [0]:
# Title case fix
df_silver = df_silver.withColumn(
    "customer_name",
    F.when(F.col("customer_name").isNull(), None)
     .otherwise(F.initcap("customer_name"))
)

In [0]:
# sanity check

df_silver.select('customer_name').distinct().show()

+--------------------+
|       customer_name|
+--------------------+
|      Fitfuel Market|
|Athlete's Choice ...|
|     Endurance Foods|
|Hydroboost Nutrition|
|Macrobite Superfoods|
|      Powersnack Hub|
|   Sprintx Nutrition|
|    Zenathlete Foods|
|Peak Performance ...|
| Primefuel Nutrition|
|       Recovery Lane|
|      Staminax Store|
|Eliteathlete Nutr...|
|      Gameplan Foods|
|   Champion's Choice|
+--------------------+



- 5: Handling missing cities

In [0]:
df_silver.filter(F.col("city").isNull()).show(truncate=False)


+-----------+-------------------+----+--------------------------+-------------+---------+
|customer_id|customer_name      |city|read_timestamp            |file_name    |file_size|
+-----------+-------------------+----+--------------------------+-------------+---------+
|789403     |Sprintx Nutrition  |NULL|2025-11-23 11:53:03.643554|customers.csv|1404     |
|789420     |Zenathlete Foods   |NULL|2025-11-23 11:53:03.643554|customers.csv|1404     |
|789521     |Primefuel Nutrition|NULL|2025-11-23 11:53:03.643554|customers.csv|1404     |
|789603     |Recovery Lane      |NULL|2025-11-23 11:53:03.643554|customers.csv|1404     |
+-----------+-------------------+----+--------------------------+-------------+---------+



In [0]:
null_customer_names = ['Sprintx Nutrition', 'Zenathlete Foods', 'Primefuel Nutrition', 'Recovery Lane']
df_silver.filter(F.col("customer_name").isin(null_customer_names)).show(truncate=False)

+-----------+-------------------+---------+--------------------------+-------------+---------+
|customer_id|customer_name      |city     |read_timestamp            |file_name    |file_size|
+-----------+-------------------+---------+--------------------------+-------------+---------+
|789401     |Sprintx Nutrition  |Bengaluru|2025-11-23 11:53:03.643554|customers.csv|1404     |
|789402     |Sprintx Nutrition  |Hyderabad|2025-11-23 11:53:03.643554|customers.csv|1404     |
|789403     |Sprintx Nutrition  |NULL     |2025-11-23 11:53:03.643554|customers.csv|1404     |
|789420     |Zenathlete Foods   |NULL     |2025-11-23 11:53:03.643554|customers.csv|1404     |
|789421     |Zenathlete Foods   |Hyderabad|2025-11-23 11:53:03.643554|customers.csv|1404     |
|789422     |Zenathlete Foods   |New Delhi|2025-11-23 11:53:03.643554|customers.csv|1404     |
|789520     |Primefuel Nutrition|Bengaluru|2025-11-23 11:53:03.643554|customers.csv|1404     |
|789521     |Primefuel Nutrition|NULL     |2025-11

In [0]:

# Business Confirmation Note: City corrections confirmed by business team
customer_city_fix = {
    # Sprintx Nutrition
    789403: "New Delhi",

    # Zenathlete Foods
    789420: "Bengaluru",

    # Primefuel Nutrition
    789521: "Hyderabad",

    # Recovery Lane
    789603: "Hyderabad"
}

df_fix = spark.createDataFrame(
    [(k, v) for k, v in customer_city_fix.items()],
    ["customer_id", "fixed_city"]
)

display(df_fix)

customer_id,fixed_city
789403,New Delhi
789420,Bengaluru
789521,Hyderabad
789603,Hyderabad


In [0]:
df_silver = (
    df_silver
    .join(df_fix, "customer_id", "left")
    .withColumn(
        "city",
        F.coalesce("city", "fixed_city")   # Replace null with fixed city
    )
    .drop("fixed_city")
)

In [0]:
# Sanity Checks

null_customer_names = ['Sprintx Nutrition', 'Zenathlete Foods', 'Primefuel Nutrition', 'Recovery Lane']
df_silver.filter(F.col("customer_name").isin(null_customer_names)).show(truncate=False)

+-----------+-------------------+---------+--------------------------+-------------+---------+
|customer_id|customer_name      |city     |read_timestamp            |file_name    |file_size|
+-----------+-------------------+---------+--------------------------+-------------+---------+
|789420     |Zenathlete Foods   |Bengaluru|2025-11-23 11:53:03.643554|customers.csv|1404     |
|789601     |Recovery Lane      |Bengaluru|2025-11-23 11:53:03.643554|customers.csv|1404     |
|789402     |Sprintx Nutrition  |Hyderabad|2025-11-23 11:53:03.643554|customers.csv|1404     |
|789603     |Recovery Lane      |Hyderabad|2025-11-23 11:53:03.643554|customers.csv|1404     |
|789401     |Sprintx Nutrition  |Bengaluru|2025-11-23 11:53:03.643554|customers.csv|1404     |
|789421     |Zenathlete Foods   |Hyderabad|2025-11-23 11:53:03.643554|customers.csv|1404     |
|789520     |Primefuel Nutrition|Bengaluru|2025-11-23 11:53:03.643554|customers.csv|1404     |
|789522     |Primefuel Nutrition|New Delhi|2025-11

- 6: Convert customer_id to string

In [0]:
df_silver = df_silver.withColumn("customer_id", F.col("customer_id").cast("string"))
print(df_silver.printSchema())

root
 |-- customer_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- read_timestamp: timestamp (nullable = true)
 |-- file_name: string (nullable = true)
 |-- file_size: long (nullable = true)

None


### Standardizing Customer Attributes to Match Parent Company Data Model

In [0]:
df_silver = (
    df_silver
    # Build final customer column: "CustomerName-City" or "CustomerName-Unknown"
    .withColumn(
        "customer",
        F.concat_ws("-", "customer_name", F.coalesce(F.col("city"), F.lit("Unknown")))
    )
    
    # Static attributes aligned with parent data model
    .withColumn("market", F.lit("India"))
    .withColumn("platform", F.lit("Sports Bar"))
    .withColumn("channel", F.lit("Acquisition"))
)

In [0]:
display(df_silver.limit(5))

customer_id,customer_name,city,read_timestamp,file_name,file_size,customer,market,platform,channel
789503,Peak Performance Store,New Delhi,2025-11-23T11:53:03.643Z,customers.csv,1404,Peak Performance Store-New Delhi,India,Sports Bar,Acquisition
789420,Zenathlete Foods,Bengaluru,2025-11-23T11:53:03.643Z,customers.csv,1404,Zenathlete Foods-Bengaluru,India,Sports Bar,Acquisition
789703,Staminax Store,New Delhi,2025-11-23T11:53:03.643Z,customers.csv,1404,Staminax Store-New Delhi,India,Sports Bar,Acquisition
789621,Eliteathlete Nutrition,Hyderabad,2025-11-23T11:53:03.643Z,customers.csv,1404,Eliteathlete Nutrition-Hyderabad,India,Sports Bar,Acquisition
789101,Endurance Foods,Bengaluru,2025-11-23T11:53:03.643Z,customers.csv,1404,Endurance Foods-Bengaluru,India,Sports Bar,Acquisition


In [0]:
df_silver.write\
 .format("delta") \
 .option("delta.enableChangeDataFeed", "true") \
 .option("mergeSchema", "true") \
 .mode("overwrite") \
 .saveAsTable(f"{catalog}.{silver_schema}.{data_source}")

## Gold

In [0]:
df_silver = spark.sql(f"SELECT * FROM {catalog}.{silver_schema}.{data_source};")


# take req cols only
# "customer_id, customer_name, city, read_timestamp, file_name, file_size, customer, market, platform, channel"
df_gold = df_silver.select("customer_id", "customer_name", "city", "customer", "market", "platform", "channel")

In [0]:
df_gold.write\
 .format("delta") \
 .option("delta.enableChangeDataFeed", "true") \
 .mode("overwrite") \
 .saveAsTable(f"{catalog}.{gold_schema}.sb_dim_{data_source}")

## Merging Data source with parent

In [0]:
delta_table = DeltaTable.forName(spark, "fmcg.gold.dim_customers")
df_child_customers = spark.table("fmcg.gold.sb_dim_customers").select(
    F.col("customer_id").alias("customer_code"),
    "customer",
    "market",
    "platform",
    "channel"
)

In [0]:
delta_table.alias("target").merge(
    source=df_child_customers.alias("source"),
    condition="target.customer_code = source.customer_code"
).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]